<a href="https://colab.research.google.com/github/varunshukla039/Reduction-of-Electricity-bills/blob/main/EcoBillOptimizer1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install necessary libraries
!pip install pytesseract
!apt-get install -y libtesseract-dev
!pip install Pillow
!apt-get update
!apt-get install -y tesseract-ocr
!pip install pandas

# Import libraries
import pytesseract
from PIL import Image
import os
import csv
from google.colab import drive
import pandas as pd
import re

# Mount Google Drive
drive.mount('/content/drive')


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 3,744 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.1 [582 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Fetched 3,744 kB in 2s (2,108 kB/s)
Selecting previously unselected package libarchive-dev:amd64.
(Reading database ... 123589 files and directories currently installed.)
Preparing to unpack .../libarchive-dev_3.6.0-1ubunt

In [7]:
# Define functions to extract text from images
def extract_text_from_image(image_path):
    """Extracts text from an image file."""
    image = Image.open(image_path)
    return pytesseract.image_to_string(image)

def process_images_in_folder(folder_path):
    """Processes all images in the specified folder."""
    extracted_data = []

    # Iterate over all files in the folder
    for file_name in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)

        # Check if the file is an image
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            print(f"Processing {file_name}...")
            text = extract_text_from_image(file_path)
            extracted_data.append({"file_name": file_name, "extracted_text": text})

    return extracted_data

def save_extracted_data_to_csv(extracted_data, output_csv_file):
    """Saves the extracted text data to a CSV file."""
    with open(output_csv_file, 'w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=["file_name", "extracted_text"])
        writer.writeheader()
        writer.writerows(extracted_data)

# Set the path to the folder containing images
folder_path = '/content/drive/MyDrive/Resumeee'  # Replace with your folder path
output_csv_file = '/content/drive/MyDrive/extracted_text.csv'  # Replace with your desired CSV file path

# Process images and save the extracted text to a CSV file
extracted_data = process_images_in_folder(folder_path)
save_extracted_data_to_csv(extracted_data, output_csv_file)

print(f"Text extracted and saved to {output_csv_file}")


Processing Screenshot 2024-08-22 151731.png...
Text extracted and saved to /content/drive/MyDrive/extracted_text.csv


In [3]:
def correct_and_extract(text):
    """Corrects common spelling errors and extracts relevant data from the text."""
    # Define corrections
    corrections = {
        'ie rrent': 'Torrent Power',
        'GIN:': 'GIN:',
        'Meter No.': 'Meter Number:',
        'T No.': 'T No.:',
        'Bill Date': 'Bill Date:',
        'Meter Serial No.': 'Meter Serial Number:',
        'Past Reading': 'Past Reading:',
        'Present Reading': 'Present Reading:',
        'Fixed Charges': 'Fixed Charges:',
        'Excess Demand Charges': 'Excess Demand Charges:',
        'Energy Charges': 'Energy Charges:',
        'Electricity Duty': 'Electricity Duty:',
        'Regulatory Surcharge 1': 'Regulatory Surcharge 1:',
        'Regulatory Surcharge 2': 'Regulatory Surcharge 2:',
        'Total Current Dues': 'Total Current Dues:',
        'Arrears': 'Arrears:',
        'Total Dues': 'Total Dues:',
        'Last Payment Date': 'Last Payment Date:',
        'Last Payment Amount': 'Last Payment Amount:',
        'Last Payment Mode': 'Last Payment Mode:',
        'Security Deposit': 'Security Deposit:',
        'Purpose': 'Purpose:'
    }

    # Replace incorrect text with corrected text
    for wrong, correct in corrections.items():
        text = text.replace(wrong, correct)

    # Extract relevant data using regular expressions
    extracted_data = {}

    def extract_field(label):
        pattern = rf'{label}\s*(\d+[\d,.]*)'
        match = re.search(pattern, text)
        return match.group(1) if match else None

    extracted_data['meter_no'] = extract_field('Meter Number:')
    extracted_data['bill_no'] = extract_field('T No.:')
    extracted_data['bill_date'] = extract_field('Bill Date:')
    extracted_data['past_reading'] = extract_field('Past Reading:')
    extracted_data['present_reading'] = extract_field('Present Reading:')
    extracted_data['fixed_charges'] = extract_field('Fixed Charges:')
    extracted_data['excess_demand_charges'] = extract_field('Excess Demand Charges:')
    extracted_data['energy_charges'] = extract_field('Energy Charges:')
    extracted_data['electricity_duty'] = extract_field('Electricity Duty:')
    extracted_data['regulatory_surcharge_1'] = extract_field('Regulatory Surcharge 1:')
    extracted_data['regulatory_surcharge_2'] = extract_field('Regulatory Surcharge 2:')
    extracted_data['total_current_dues'] = extract_field('Total Current Dues:')
    extracted_data['arrears'] = extract_field('Arrears:')
    extracted_data['total_dues'] = extract_field('Total Dues:')
    extracted_data['last_payment_date'] = extract_field('Last Payment Date:')
    extracted_data['last_payment_amount'] = extract_field('Last Payment Amount:')
    extracted_data['last_payment_mode'] = extract_field('Last Payment Mode:')
    extracted_data['security_deposit'] = extract_field('Security Deposit:')
    extracted_data['purpose'] = extract_field('Purpose:')

    return extracted_data

# Define the path to the CSV file and output folder
csv_file_path = '/content/drive/MyDrive/extracted_text.csv'  # This should be the file generated in Step 2
output_folder = '/content/drive/MyDrive/'  # Folder to save processed data
os.makedirs(output_folder, exist_ok=True)

# Load the CSV file
df = pd.read_csv(csv_file_path)

# List to hold processed data
processed_data = []

# Process each row in the CSV file
for _, row in df.iterrows():
    text = row['extracted_text']
    corrected_data = correct_and_extract(text)
    processed_data.append(corrected_data)

# Convert the processed data to a DataFrame
processed_df = pd.DataFrame(processed_data)

# Save the processed data to a new CSV file
processed_csv_file = os.path.join(output_folder, 'processed_text_data.csv')
processed_df.to_csv(processed_csv_file, index=False, encoding='utf-8')
print(f"Processed data saved to {processed_csv_file}")


Processed data saved to /content/drive/MyDrive/processed_text_data.csv
